In [1]:
import numpy as np
import matplotlib.pyplot as plt 
from IPython.display import Math, Latex # for latex equations
from IPython.core.display import Image # for displaying images
from scipy import stats
from scipy import optimize
import networkx as nx
import pandas as pd
from pyvis.network import Network
import collections
import itertools

In [48]:
# Load data from the respective csv and construct a static graph/network of plant tissue with the state 'state'

def create_tissue(csv_path,edge_weight):
    
    csv_df = pd.read_csv(csv_path) 
    
    g = nx.Graph()
    node_list = list(sorted(set(csv_df.iloc[:,0])))
    g.add_nodes_from(node_list)
    for i in range(len(csv_df)):
        g.add_edge(csv_df.iloc[i,0],csv_df.iloc[i,1])
    initial_state = np.zeros(len(g.nodes()), dtype = np.int8)
    assert len(initial_state) == len(g.nodes())
    nx.set_node_attributes(g, dict(zip(g.nodes(), initial_state)), name="state") 
    if edge_weight == True:
        weights = [csv_df.iloc[i,2] for i in range(len(csv_df))]
        nx.set_edge_attributes(g, dict(zip(g.edges(), weights)), "weight")
    
    return g


# Update the network; basically the state of the network according to the rule-3 (r3) which corresponds with Doug's model-3 

def update_rule3(g, temp, noise_cold, noise_warm): 
    for x in list(g.nodes()):
        if temp == 0 and g.nodes[x]["state"] == 0:
            c1 = np.random.choice(['noise','no_noise'], p = [noise_cold,1-noise_cold])
            if c1 == 'noise':
                g.nodes[x]["state"] = 1
            if c1 == 'no_noise':
                neighbor_states = np.array([g.nodes[y]["state"] for y in list(g.neighbors(x))])
                if np.sum(neighbor_states)/len(neighbor_states) >= 0.5:
                    g.nodes[x]["state"] = 1

        if temp == 1 and g.nodes[x]["state"] == 1:
            c2 = np.random.choice(['noise','no_noise'], p = [noise_warm,1-noise_warm])
            if c2 == 'noise':
                g.nodes[x]["state"] = 0
                
    return g          
    
def update_spontaneous(g, jump_state):
    if jump_state == "default":
        jump = np.zeros((len(g.nodes()),),dtype = int)
    else:
        assert len(g.nodes()) == len(jump_state)
        jump = jump_state
    for i in range(len(g.nodes())):
        g.nodes[list(g.nodes())[i]]["state"] = jump[i]
        


def trajectory(g, temp_sch, noise_cold, noise_warm): # temp schedule is the list of cold (0), warm(1) schedules 
    
    temp_array = np.array([],dtype = int)         
    for i in range(len(temp_sch)):
        if int(i/2)*2  == i: 
            to_append = np.zeros(temp_sch[i], dtype = int)
        else:
            to_append = np.ones(temp_sch[i], dtype = int)
        temp_array = np.append(temp_array, to_append)
        
    
    trajectory = np.empty([len(temp_array)+1 , len(g.nodes())],dtype = int)
    g_0 = g
    trajectory[0] = np.array([g_0.nodes[j]["state"] for j in g_0.nodes()])
    for k in range(len(temp_array)):
        g_0 = update_rule3(g_0, temp_array[k], noise_cold, noise_warm)
        trajectory[k+1] = np.array([g_0.nodes[j]["state"] for j in g_0.nodes()])
        
    time_array = np.arange(len(temp_array)+1)
    avg_exp = []
    for i in range(len(trajectory)):
        avg_exp.append(np.sum(trajectory[i])*100/len(trajectory[0]))
    expression_level = np.array(avg_exp)
    
    trajectory_df = pd.DataFrame(trajectory, columns = ['node{}'.format(x) for x in g.nodes()])
    trajectory_df.insert(0, "Time_step", time_array)
    trajectory_df.insert(1, "expression_level", expression_level)
    return trajectory_df
    

def plt_trajectory(g, temp_sch, noise_cold, noise_warm, ensemble_size):
    plt_data = trajectory(g, temp_sch, noise_cold, noise_warm).iloc[:,[0,1]]
    plt_data.rename(columns={"expression_level":"Sim_1"} ,inplace=True)
    for i in range(ensemble_size-1):
        update_spontaneous(g, "default")
        traj = trajectory(g, temp_sch, noise_cold, noise_warm).iloc[:,1]
        plt_data.insert(i+2,"Sim_{}".format(i+2), traj)
    
    return plt_data
        


#def plt_tissue(g,edge_weight)


In [121]:

T = create_tissue("../data/exp_raw/sam.csv",edge_weight = False)

# plt.figure(figsize = (10,10))
# pos = nx.kamada_kawai_layout(T)
# nx.draw_networkx_nodes(T,pos,node_color='lightgreen')
# nx.draw_networkx_edges(T,pos,edge_color = 'blue')

In [122]:
temp_sch = np.array([10,10,10,10,10,10])
ensemble_size = 20
Tr = plt_trajectory(T,temp_sch, 0.01, 0.03, ensemble_size)
Tr['mean'] = Tr.iloc[:,1:5].mean(axis=1)
Tr['std'] = Tr.iloc[:,1:5].std(axis=1)
Tr['upper'] = Tr.iloc[:,21] + 0.5*Tr.iloc[:,22]
Tr['lower'] = Tr.iloc[:,21] - 0.5*Tr.iloc[:,22]
Tr

,Time_step,Sim_1,Sim_2,Sim_3,Sim_4,Sim_5,Sim_6,Sim_7,Sim_8,Sim_9,...,Sim_15,Sim_16,Sim_17,Sim_18,Sim_19,Sim_20,mean,std,upper,lower
0,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,1,0.354610,0.709220,1.063830,1.063830,1.773050,0.709220,1.063830,1.063830,0.709220,...,1.063830,0.709220,1.063830,1.063830,0.709220,0.000000,0.797872,0.339513,0.967629,0.628116
2,2,1.418440,0.709220,2.127660,2.127660,2.836879,1.773050,2.127660,2.127660,2.127660,...,1.418440,2.127660,2.127660,2.127660,1.418440,0.354610,1.595745,0.679026,1.935258,1.256232
3,3,1.773050,1.418440,3.191489,2.482270,3.546099,3.191489,2.836879,3.191489,3.900709,...,2.836879,3.546099,3.546099,2.836879,1.773050,1.418440,2.216312,0.786296,2.609460,1.823164
4,4,3.191489,1.418440,3.900709,3.191489,4.609929,5.319149,3.191489,3.900709,6.028369,...,2.836879,5.319149,4.609929,5.319149,2.482270,2.127660,2.925532,1.058893,3.454979,2.396085
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56,56,71.276596,41.134752,82.269504,80.851064,49.645390,56.382979,54.609929,80.141844,57.446809,...,48.226950,51.773050,54.609929,56.382979,53.546099,72.695035,68.882979,19.132238,78.449098,59.316860
57,57,68.439716,40.780142,78.723404,79.787234,48.226950,54.609929,53.191489,78.368794,56.382979,...,46.808511,49.645390,52.482270,54.609929,51.418440,70.567376,66.932624,18.170371,76.017810,57.847439
58,58,64.893617,40.070922,76.241135,75.531915,46.453901,52.836879,51.418440,75.531915,55.673759,...,45.390071,48.581560,51.063830,51.418440,50.000000,68.085106,64.184397,16.892738,72.630766,55.738028
59,59,63.120567,39.007092,74.113475,74.822695,45.744681,51.418440,50.709220,73.049645,53.546099,...,43.617021,47.872340,49.645390,50.709220,47.872340,66.666667,62.765957,16.720651,71.126283,54.405632


In [129]:
import altair as alt
line = alt.Chart(Tr).mark_line().encode(
    x=alt.X('Time_step', title='Time [1 unit = 1 day]'),
    y=alt.Y('mean', title= 'Expression level (%) of the tissue')
)

band = alt.Chart(Tr).mark_area(
    opacity=0.4, color='green').encode(
    x=alt.X('Time_step', title='Time [1 unit = 1 day]'),
    y='lower',
    y2='upper'
)
(line + band).interactive()

alt.LayerChart(...)